In [6]:
# -*- coding: utf-8 -*-
"""
Created on Mon Apr  4 23:28:32 2022

@author: parkh
"""

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import gym
import collections
import random
from SIM_gannt4 import *

learning_rate = 0.001  
gamma = 0.99
buffer_limit = 10000
batch_size = 16

class ReplayBuffer():
    def __init__(self):
        self.buffer = collections.deque(maxlen=buffer_limit);
    def put(self, transition):
        self.buffer.append(transition)
    def sample(self, n):
        mini_batch = random.sample(self.buffer, n)
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [],[],[],[],[]
        
        for transition in mini_batch:
            s, a, r, s_prime, done_mask = transition
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            done_mask_lst.append([done_mask])
            
        return torch.tensor(s_lst, dtype=torch. float),torch.tensor(a_lst), torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch. float), torch.tensor(done_mask_lst)
    
    def size(self):
        return len(self.buffer)

class Qnet(nn.Module):
    def __init__(self):
        super(Qnet, self).__init__()
        self.fc1 = nn.Linear(132,64)
        self.fc2 = nn.Linear(64,32)
        self.fc3 = nn.Linear(32,6)
        self.number_of_time_list = np.array([1 for x in range(6)])
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def sample_action(self, obs, epsilon):
        out = self.forward(obs)
        #print(out)
        out2 = out.detach().numpy()
        act_list = out2/self.number_of_time_list
        act = np.argmax(act_list)
        coin = random.random()
        if coin < epsilon:
            return random.randint(0, 5)
        else:
            return act
    def select_action(self, obs, epsilon):
        out = self.forward(obs)
        out2 = out.detach().numpy()
        act_list = out2/self.number_of_time_list
        act = np.argmax(act_list)
        return act,act_list
        
def train(q, q_target, memory, optimizer):
    for i in range(10):
        s,a,r,s_prime,done_mask = memory.sample(batch_size)
        #q.number_of_time_list[a] += 1    
        q_out = q(s)
        q_a = q_out.gather(1,a)
        max_q_prime = q_target(s_prime).max (1)[0].unsqueeze(1)
        target = r + gamma * max_q_prime * done_mask
        loss = F.smooth_l1_loss(q_a, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
            
def main():
    env = FJSP_simulator('C:/Users\parkh/git_tlsgudcks/simulator/data/FJSP_SIM7_all.csv','C:/Users/parkh/FJSP_SETUP_SIM.csv',1)
    q = Qnet()
    q_target = Qnet()
    q_target.load_state_dict(q.state_dict())
    memory = ReplayBuffer()
    print_interval = 1
    q_load = 20
    score = 0.0
    optimizer = optim.Adam(q.parameters(), lr=learning_rate)
    
    for n_epi in range(1000):
        epsilon = max(0.01 , 0.08 - 0.02*(n_epi/200))
        s = env.reset()
        done = False
        score = 0.0
        while not done:
            a = q.sample_action(torch.from_numpy(s). float(), epsilon)
            s_prime, r, done = env.step(a)
            done_mask =0.0 if done else 1.0
            if done == False:
                memory.put((s,a,r,s_prime,done_mask))
                s = s_prime
                score += r
            if done:
                break
        if memory.size()>1000:
            train(q, q_target, memory, optimizer)
            
        if n_epi % print_interval==0 and n_epi!=0:
            #q_target.load_state_dict(q.state_dict())
            Flow_time, machine_utill ,util, makespan = env.performance_measure()
            print("--------------------------------------------------")
            print("flow time: {}, util : {:.3f}, makespan : {}".format(Flow_time, util, makespan))
            print("n_episode: {}, score : {:.1f}, n_buffer : {}, eps : {:.1f}%".format(n_epi, score/print_interval,memory.size(),epsilon*100))
            #score=0.0
        if n_epi % q_load ==0 and n_epi!=0:
            q_target.load_state_dict(q.state_dict())
        s = env.reset()
        done = False
        score = 0.0
    while not done:
        a, a_list = q.select_action(torch.from_numpy(s). float(), epsilon)
        print(a_list)
        print(a)
        s_prime, r, done = env.step(a)
        print(r)
        s = s_prime
        score += r
        if done:
            break
    Flow_time, machine_util, util, makespan = env.performance_measure()
    env.gannt_chart()
    return Flow_time, machine_util, util, makespan, score
Flow_time, machine_util, util, makespan, score =main()
print("FlowTime:" , Flow_time)
print("machine_util:" , machine_util)
print("util:" , util)
print("makespan:" , makespan)
print("Score" , score)
      
    

--------------------------------------------------
flow time: 3853, util : 0.801, makespan : 232
n_episode: 1, score : -277.0, n_buffer : 227, eps : 8.0%
--------------------------------------------------
flow time: 3970, util : 0.790, makespan : 240
n_episode: 2, score : -303.0, n_buffer : 342, eps : 8.0%
--------------------------------------------------
flow time: 3649, util : 0.819, makespan : 232
n_episode: 3, score : -252.0, n_buffer : 457, eps : 8.0%
--------------------------------------------------
flow time: 3791, util : 0.795, makespan : 229
n_episode: 4, score : -281.0, n_buffer : 570, eps : 8.0%
--------------------------------------------------
flow time: 3734, util : 0.823, makespan : 230
n_episode: 5, score : -244.0, n_buffer : 685, eps : 8.0%
--------------------------------------------------
flow time: 4045, util : 0.806, makespan : 245
n_episode: 6, score : -285.0, n_buffer : 801, eps : 7.9%
--------------------------------------------------
flow time: 3823, util : 0

--------------------------------------------------
flow time: 3063, util : 0.825, makespan : 247
n_episode: 54, score : -260.0, n_buffer : 7521, eps : 7.5%
--------------------------------------------------
flow time: 2914, util : 0.806, makespan : 277
n_episode: 55, score : -323.0, n_buffer : 7666, eps : 7.4%
--------------------------------------------------
flow time: 2793, util : 0.803, makespan : 286
n_episode: 56, score : -338.0, n_buffer : 7823, eps : 7.4%
--------------------------------------------------
flow time: 2758, util : 0.775, makespan : 288
n_episode: 57, score : -388.0, n_buffer : 7974, eps : 7.4%
--------------------------------------------------
flow time: 3017, util : 0.824, makespan : 288
n_episode: 58, score : -304.0, n_buffer : 8113, eps : 7.4%
--------------------------------------------------
flow time: 2870, util : 0.813, makespan : 285
n_episode: 59, score : -320.0, n_buffer : 8258, eps : 7.4%
--------------------------------------------------
flow time: 26

--------------------------------------------------
flow time: 2873, util : 0.749, makespan : 300
n_episode: 107, score : -451.0, n_buffer : 10000, eps : 6.9%
--------------------------------------------------
flow time: 2908, util : 0.786, makespan : 301
n_episode: 108, score : -386.0, n_buffer : 10000, eps : 6.9%
--------------------------------------------------
flow time: 2832, util : 0.826, makespan : 271
n_episode: 109, score : -283.0, n_buffer : 10000, eps : 6.9%
--------------------------------------------------
flow time: 2749, util : 0.759, makespan : 300
n_episode: 110, score : -434.0, n_buffer : 10000, eps : 6.9%
--------------------------------------------------
flow time: 2668, util : 0.813, makespan : 269
n_episode: 111, score : -302.0, n_buffer : 10000, eps : 6.9%
--------------------------------------------------
flow time: 2706, util : 0.798, makespan : 279
n_episode: 112, score : -338.0, n_buffer : 10000, eps : 6.9%
--------------------------------------------------
f

--------------------------------------------------
flow time: 3310, util : 0.833, makespan : 270
n_episode: 159, score : -270.0, n_buffer : 10000, eps : 6.4%
--------------------------------------------------
flow time: 3350, util : 0.762, makespan : 293
n_episode: 160, score : -419.0, n_buffer : 10000, eps : 6.4%
--------------------------------------------------
flow time: 2822, util : 0.862, makespan : 265
n_episode: 161, score : -220.0, n_buffer : 10000, eps : 6.4%
--------------------------------------------------
flow time: 2802, util : 0.777, makespan : 283
n_episode: 162, score : -378.0, n_buffer : 10000, eps : 6.4%
--------------------------------------------------
flow time: 3009, util : 0.783, makespan : 287
n_episode: 163, score : -374.0, n_buffer : 10000, eps : 6.4%
--------------------------------------------------
flow time: 2865, util : 0.763, makespan : 293
n_episode: 164, score : -417.0, n_buffer : 10000, eps : 6.4%
--------------------------------------------------
f

--------------------------------------------------
flow time: 2876, util : 0.800, makespan : 262
n_episode: 211, score : -314.0, n_buffer : 10000, eps : 5.9%
--------------------------------------------------
flow time: 2794, util : 0.776, makespan : 293
n_episode: 212, score : -393.0, n_buffer : 10000, eps : 5.9%
--------------------------------------------------
flow time: 2937, util : 0.805, makespan : 274
n_episode: 213, score : -321.0, n_buffer : 10000, eps : 5.9%
--------------------------------------------------
flow time: 2880, util : 0.879, makespan : 262
n_episode: 214, score : -190.0, n_buffer : 10000, eps : 5.9%
--------------------------------------------------
flow time: 2652, util : 0.768, makespan : 275
n_episode: 215, score : -383.0, n_buffer : 10000, eps : 5.9%
--------------------------------------------------
flow time: 2871, util : 0.779, makespan : 272
n_episode: 216, score : -361.0, n_buffer : 10000, eps : 5.8%
--------------------------------------------------
f

...

--------------------------------------------------
flow time: 2769, util : 0.793, makespan : 282
n_episode: 887, score : -350.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 3192, util : 0.761, makespan : 312
n_episode: 888, score : -447.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 3055, util : 0.761, makespan : 310
n_episode: 889, score : -445.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 2645, util : 0.798, makespan : 280
n_episode: 890, score : -340.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 2649, util : 0.797, makespan : 285
n_episode: 891, score : -347.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 2742, util : 0.794, makespan : 287
n_episode: 892, score : -354.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
f

--------------------------------------------------
flow time: 2795, util : 0.787, makespan : 285
n_episode: 939, score : -364.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 2626, util : 0.830, makespan : 264
n_episode: 940, score : -270.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 2622, util : 0.813, makespan : 267
n_episode: 941, score : -300.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 2771, util : 0.788, makespan : 293
n_episode: 942, score : -373.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 2752, util : 0.739, makespan : 299
n_episode: 943, score : -468.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 2655, util : 0.753, makespan : 289
n_episode: 944, score : -429.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
f

--------------------------------------------------
flow time: 2655, util : 0.801, makespan : 285
n_episode: 991, score : -340.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 2693, util : 0.815, makespan : 273
n_episode: 992, score : -303.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 2649, util : 0.797, makespan : 285
n_episode: 993, score : -347.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 2666, util : 0.807, makespan : 277
n_episode: 994, score : -321.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 2619, util : 0.801, makespan : 274
n_episode: 995, score : -327.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 2649, util : 0.797, makespan : 285
n_episode: 996, score : -347.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
f

0
[-70.79855347 -72.71060944 -69.63954163 -70.87245178 -70.93209076
 -69.96617126]
2
-2
[-71.8680954  -73.81074524 -70.72930145 -71.94762421 -72.03804016
 -71.02929688]
2
0
[-79.87005615 -82.07470703 -78.50627136 -80.00773621 -80.02365875
 -79.10943604]
2
0
[-81.89839935 -84.14237213 -80.5105896  -81.98932648 -82.11592865
 -81.18302155]
2
0
[-86.55684662 -88.96447754 -85.0418396  -86.67524719 -86.82202148
 -85.90972137]
2
-2
[-93.75437164 -96.31983185 -91.97227478 -93.79595947 -93.98214722
 -93.09033966]
2
0
[ -98.04298401 -100.73561096  -96.16830444  -98.11254883  -98.25631714
  -97.47563934]
2
-5
[ -98.28897095 -100.98470306  -96.41146088  -98.35029602  -98.51000214
  -97.7259903 ]
2
-5
[-100.23821259 -102.96954346  -98.27378082 -100.27056122 -100.43655396
  -99.6656723 ]
2
-5
[-100.48419189 -103.21864319  -98.51693726 -100.50831604 -100.69023895
  -99.91603088]
2
-4
[-103.30065918 -106.0929184  -101.25095367 -103.26564789 -103.56845093
 -102.78146362]
2
-1
[-103.69050598 -106.489883

FlowTime: 3050
machine_util: 0
util: 0.818892900120337
makespan: 277
Score -301.0
